In [1]:
import sys
sys.path.append('../src')
sys.path.append('../eval')
sys.path.append('../envs/misyak/')

In [2]:
import numpy as np
import pandas as pd
from collections import OrderedDict

from GenerativeSignaler import SignalerZero
from consistentSignalChecks_Misyak import signalIsConsistent_Boxes
from OverloadedReceiver import ReceiverZero
from OverloadedSignaler import SignalerOne
from misyakConstruction import *
from mindConstruction import *
from evaluateOverloadedSignaling import *

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches 

# Main Setup

In [5]:
removeZeroes = True
#world spaces
twoRewardWorldSpace = getWorldSpace(wall = False, nBoxes = 3, nRewards = 2)
oneRewardWorldSpace = getWorldSpace(wall = False, nBoxes = 3, nRewards = 1)
wallWorldSpace = getWorldSpace(wall = True, nBoxes = 3, nRewards = 2)

#action spaces
oneAxActionSpace = getActionSpace(nBoxes = 3, nReceiverChoices = 1)
twoAxActionSpace = getActionSpace(nBoxes = 3, nReceiverChoices = 2)

#signal spaces
oneTokenSignalSpace = getSignalSpace(nBoxes=3, nSignals = 1)
twoTokenSignalSpace = getSignalSpace(nBoxes=3, nSignals = 2)

if removeZeroes:
    oneAxActionSpace.remove((0,0,0))
    twoAxActionSpace.remove((0,0,0))
    oneTokenSignalSpace.remove((0,0,0))
    twoTokenSignalSpace.remove((0,0,0))

#condition common ground spaces
twoTokenCondition = {'worlds': twoRewardWorldSpace, 'desires': [1], 'intentions': [1], 'actions': twoAxActionSpace, 'signals':twoTokenSignalSpace}
inversionCondition = {'worlds': twoRewardWorldSpace, 'desires': [1], 'intentions': [1], 'actions': twoAxActionSpace, 'signals':oneTokenSignalSpace}
oneAxCondition = {'worlds': twoRewardWorldSpace, 'desires': [1], 'intentions': [1], 'actions': oneAxActionSpace, 'signals':oneTokenSignalSpace}
wallCondition = {'worlds': wallWorldSpace, 'desires': [1], 'intentions': [1], 'actions': twoAxActionSpace, 'signals':oneTokenSignalSpace}

conditionParameters = {'twoToken': twoTokenCondition,'inversion': inversionCondition,'wall': wallCondition,'oneAx': oneAxCondition}

signalerHumanResults = {'inversion': .91, 'wall': .48, 'wall_goto': .5, 'oneAx':.32, 'oneAx_goto': .68, 'twoToken':.02}
signalerHumanResultsExperiment2 = {'twoToken':0.0, 'twoToken_goto': 1.0, 'inversion':.56, 'inversion_goto':.42, 'wall':.23, 'wall_goto': .77, 'oneAx':.375, 'oneAx_goto': .625}

fixedParameters = {'locationCost':0, 'rewardValue':5, 'nonRewardCost':0, 'observation': {'worlds':(1,1,0)}}

# Tuning Variable Grids

## Experiment 1


In [6]:
manipulatedVariables = OrderedDict()
manipulatedVariables['alpha'] = list(np.linspace(1.0,15.0,16))
manipulatedVariables['signalTypePriors'] = list(np.linspace(.35, .7, 16))

In [7]:
levelNames = list(manipulatedVariables.keys())
levelValues = list(manipulatedVariables.values())
modelIndex = pd.MultiIndex.from_product(levelValues, names=levelNames)
toSplitFrame = pd.DataFrame(index=modelIndex)

computeStatistics =  EvaluateOverloadedSignaler(conditionDictionary = conditionParameters, targetResult = signalerHumanResults, fixedParameters=fixedParameters)
computeConditionStatistics = lambda condition: computeStatistics(condition.index.get_level_values('alpha')[0], condition.index.get_level_values('signalTypePriors')[0])
conditionStatistics_16x16 = toSplitFrame.groupby(levelNames).apply(computeConditionStatistics)

KeyError: 'twoToken_goto'

In [ ]:
conditionStatistics_16x16

In [ ]:
conditionStatistics_16x16.to_csv("expt1_16x16conditions.txt",sep="|")

In [ ]:
conditionStatistics_16x16.loc[conditionStatistics_16x16['total_rmse'] < .15]

### Smaller Experiment 1 Grid, chosen from large parameter search


In [8]:
manipulatedVariables = OrderedDict()
manipulatedVariables['alpha'] = [5.5,6, 6.5,7, 7.5]
manipulatedVariables['signalTypePriors'] = [.45, .47, .49, .51]

In [9]:
levelNames = list(manipulatedVariables.keys())
levelValues = list(manipulatedVariables.values())
modelIndex = pd.MultiIndex.from_product(levelValues, names=levelNames)
toSplitFrame = pd.DataFrame(index=modelIndex)

computeStatistics =  EvaluateOverloadedSignaler(conditionDictionary = conditionParameters, targetResult = signalerHumanResults, fixedParameters=fixedParameters)
computeConditionStatistics = lambda condition: computeStatistics(condition.index.get_level_values('alpha')[0], condition.index.get_level_values('signalTypePriors')[0])
conditionStatistics_4x5FromLarge = toSplitFrame.groupby(levelNames).apply(computeConditionStatistics)

KeyError: 'twoToken_goto'

In [ ]:
conditionStatistics_4x5FromLarge

## Experiment 2


### Large Grid
- 16 by 16 
- over a wide range of possibilities

In [ ]:
manipulatedVariables = OrderedDict()
manipulatedVariables['alpha'] = list(np.linspace(1.0,15.0,16))
manipulatedVariables['signalTypePriors'] = list(np.linspace(.35, .7, 16))

In [ ]:
levelNames = list(manipulatedVariables.keys())
levelValues = list(manipulatedVariables.values())
modelIndex = pd.MultiIndex.from_product(levelValues, names=levelNames)
toSplitFrame = pd.DataFrame(index=modelIndex)

computeStatistics =  EvaluateOverloadedSignaler(conditionDictionary = conditionParameters, targetResult = signalerHumanResultsExperiment2, fixedParameters=fixedParameters)
computeConditionStatistics = lambda condition: computeStatistics(condition.index.get_level_values('alpha')[0], condition.index.get_level_values('signalTypePriors')[0])
conditionStatistics_16x16Expt2 = toSplitFrame.groupby(levelNames).apply(computeConditionStatistics)

In [ ]:
conditionStatistics_16x16Expt2

In [ ]:
conditionStatistics_16x16Expt2.loc[conditionStatistics_16x16Expt2['total_rmse'] < .08]

### Large Grid (Fine)

Chosen from the large grid over a large set of possibilities

- 12 by 12
- Testing for all condition matches, including null signaling

In [ ]:
manipulatedVariables = OrderedDict()
manipulatedVariables['alpha'] = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7]
manipulatedVariables['signalTypePriors'] = [.49, .5, .51, .52, .53, .54, .55, .56, .57, .58, .59, .6]

In [ ]:
levelNames = list(manipulatedVariables.keys())
levelValues = list(manipulatedVariables.values())
modelIndex = pd.MultiIndex.from_product(levelValues, names=levelNames)
toSplitFrame = pd.DataFrame(index=modelIndex)

computeStatistics =  EvaluateOverloadedSignaler(conditionDictionary = conditionParameters, targetResult = signalerHumanResultsExperiment2, fixedParameters=fixedParameters)
computeConditionStatistics = lambda condition: computeStatistics(condition.index.get_level_values('alpha')[0], condition.index.get_level_values('signalTypePriors')[0])
conditionStatistics_12x12Expt2 = toSplitFrame.groupby(levelNames).apply(computeConditionStatistics)

In [ ]:
conditionStatistics_12x12Expt2

In [ ]:
conditionStatistics_12x12Expt2.loc[conditionStatistics_12x12Expt2['total_rmse'] < .11]

### Optimal Parameter grid - Null Full PDF
- for Experiment 2, null signaling and null actions included, odd one out proportion and go to signaling (full signal PDF) as metric

In [ ]:
manipulatedVariables = OrderedDict()
manipulatedVariables['alpha'] = [3.0, 3.5, 4, 4.5, 5]
manipulatedVariables['signalTypePriors'] = [.55, .56, .57, .58]

In [ ]:
levelNames = list(manipulatedVariables.keys())
levelValues = list(manipulatedVariables.values())
modelIndex = pd.MultiIndex.from_product(levelValues, names=levelNames)
toSplitFrame = pd.DataFrame(index=modelIndex)

computeStatistics =  EvaluateOverloadedSignaler(conditionDictionary = conditionParameters, targetResult = signalerHumanResultsExperiment2, fixedParameters=fixedParameters)
computeConditionStatistics = lambda condition: computeStatistics(condition.index.get_level_values('alpha')[0], condition.index.get_level_values('signalTypePriors')[0])
conditionStatistics_5x4Expt2 = toSplitFrame.groupby(levelNames).apply(computeConditionStatistics)

In [ ]:
conditionStatistics_5x4Expt2

### Optimal Parameter grid - No Null OOOP
 - for Experiment 2, null signaling and null actions removed, odd one out proportion as metric

In [ ]:
manipulatedVariables = OrderedDict()
manipulatedVariables['alpha'] = [1, 1.5, 1.75, 2, 2.5, 3]
manipulatedVariables['signalTypePriors'] = [.55, .57, .59,.6 ,.61, .63]

In [ ]:
levelNames = list(manipulatedVariables.keys())
levelValues = list(manipulatedVariables.values())
modelIndex = pd.MultiIndex.from_product(levelValues, names=levelNames)
toSplitFrame = pd.DataFrame(index=modelIndex)

computeStatistics =  EvaluateOverloadedSignaler(conditionDictionary = conditionParameters, targetResult = signalerHumanResultsExperiment2, fixedParameters=fixedParameters)
computeConditionStatistics = lambda condition: computeStatistics(condition.index.get_level_values('alpha')[0], condition.index.get_level_values('signalTypePriors')[0])
optimalNoZeroes = toSplitFrame.groupby(levelNames).apply(computeConditionStatistics)

In [ ]:
optimalNoZeroes.loc[optimalNoZeroes['total_rmse'] < .04]

### Optimal Parameter Grid - Null OOOP
- For experiment 2, null signaling and actions included, odd one out proportion as metric

In [ ]:
manipulatedVariables = OrderedDict()
manipulatedVariables['alpha'] = [6,8,10,12,14]
manipulatedVariables['signalTypePriors'] = [.49, .5, .51, .52, .53, .54]

# Plotting

### Hyperparameters of Plotting

In [ ]:
#experiment 1 target human performances from paper
targetMeansExp1 = {'twoToken':0.02, 'inversion':.91, 'wall':.48, 'oneAx':.32}
targetSEExp1 = {'twoToken':0.025, 'inversion':.042, 'wall':.073, 'oneAx':.082}

#experiment 2 target human performances from paper
targetMeansExp2 = {'twoToken':0.0, 'twoToken_goto':1.0,'inversion':.56,'inversion_goto':.42, 'wall':.23,'wall_goto':.77, 'oneAx':.375, 'oneAx_goto':.625}
targetSEExp2 = {'twoToken':0.0, 'twoToken_goto':0.0,'inversion':.1,'inversion_goto':.1, 'wall':.08,'wall_goto':.08, 'oneAx':.1, 'oneAx_goto':.1}

In [ ]:
#plotting hyperparameters
figureSize = (25,18)
horizontalSpaceBetweenSubplots = .45
legendColors = {'twoToken' : 'thistle','twoToken_goto': 'thistle', 'inversion':'royalblue', 'inversion_goto':'royalblue','wall':'gold','wall_goto':'gold','oneAx':'lightsalmon', 'oneAx_goto':'lightsalmon'}
hatch = {'twoToken':'n', 'twoToken_goto':'//','inversion':'n','inversion_goto':'//', 'wall':'n','wall_goto':'//', 'oneAx':'n', 'oneAx_goto':'//'}

### Specific to dataFrame

In [ ]:
#dataframe to plot
dataframeName = conditionStatistics_5x4Expt2
dataFrameRowLength = len(manipulatedVariables['alpha'])
dataFrameColLength = len(manipulatedVariables['signalTypePriors'])

### Overarching Plotting Code

In [ ]:
fig = plt.figure(figsize=figureSize)
numRows = dataFrameRowLength
numColumns = dataFrameColLength
plotCounter = 1

for alpha, grp in dataframeName.groupby('alpha'):
    grp.index = grp.index.droplevel('alpha')
    for signalTypePriors, group in grp.groupby('signalTypePriors'):

        #subplot axis labels
        axForDraw = fig.add_subplot(numRows, numColumns, plotCounter)
        if plotCounter % numColumns == 1:
            axForDraw.set_ylabel('alpha: {}'.format(alpha))
        if plotCounter <= numColumns:
            axForDraw.set_title('p(c) = 1: {}'.format(round(signalTypePriors, 2)))
        axForDraw.set_ylim(0, 1)

        #draw subplot
        drawPerformanceBarPlot(group, signalTypePriors, axForDraw, targetMeansExp2, targetSEExp2, legendColors, 8, hatch)

        plotCounter += 1

#Title
plt.suptitle('Proportion Usage of Odd One Out Signaling')

#legend
legend_dict = legendColors
patchList = []
trialList = ["twoToken", 'inversion', 'wall', 'oneAx']
for key in trialList:
    data_key = mpatches.Patch(color=legend_dict[key], label=key, hatch = hatch[key])
    patchList.append(data_key)
plt.legend(handles=patchList, loc='best')

#adjust horizontal spacing of subplots
fig.subplots_adjust(wspace=horizontalSpaceBetweenSubplots)

plt.show()

## Optimal Parameter Plotting - single plot from data fram

In [ ]:
def drawPerformanceBarPlot(dfCondition, oddOneOutConditions, targetSEDictionary, conditionColors, nConditions = 4, hatches = None, barWidth = .35):
    plotTitle = 'Proportion of Odd-One-Out Signaling Usage'
    fig = plt.figure(figsize=figureSize)
    fig, ax = plt.subplots() 
    N = nConditions
    ind = np.arange(N)  # the x locations for the groups
    width = barWidth      # the width of the bars

    modelMeans = []
    targetMeans = []
    targetSEs = []
    cols = []
    hatchPattern = []

    for condition, trgtMean in oddOneOutConditions.items():
        modelMeans.append(dfCondition[condition].values[0])
        targetMeans.append(trgtMean)
        targetSEs.append(targetSEDictionary[condition])
        cols.append(conditionColors[condition])

        if hatches is None:
            hatchPattern.append('n')
        else:
            hatchPattern.append(hatches[condition])

    modelPreds = ax.bar(ind, tuple(modelMeans), width, color=cols, alpha = .5)
    
    humanPreds = ax.bar(ind + width, tuple(targetMeans), width, color=cols, yerr=tuple(targetSEs))

    for modelPred, humanPred, pattern in zip(modelPreds, humanPreds, hatchPattern):
        modelPred.set_hatch(pattern)
        humanPred.set_hatch(pattern)
    
    ax.set_ylim(0, 1)
    ax.set_yticks([0,.5,1])
    ax.set_xticks([])
    fig.suptitle(plotTitle)
    plt.show()

In [ ]:
#experiment 2 target human performances from paper
targetMeansExp2 = {'twoToken':0.0, 'twoToken_goto':1.0,'inversion':.56,'inversion_goto':.42, 'wall':.23,'wall_goto':.77, 'oneAx':.375, 'oneAx_goto':.625}
targetSEExp2 = {'twoToken':0.0, 'twoToken_goto':0.0,'inversion':.1,'inversion_goto':.1, 'wall':.08,'wall_goto':.08, 'oneAx':.1, 'oneAx_goto':.1}
#plotting hyperparameters
figureSize = (25,18)
horizontalSpaceBetweenSubplots = .45
legendColors = {'twoToken' : 'thistle','twoToken_goto': 'thistle', 'inversion':'royalblue', 'inversion_goto':'royalblue','wall':'gold','wall_goto':'gold','oneAx':'lightsalmon', 'oneAx_goto':'lightsalmon'}
hatch = {'twoToken':'n', 'twoToken_goto':'//','inversion':'n','inversion_goto':'//', 'wall':'n','wall_goto':'//', 'oneAx':'n', 'oneAx_goto':'//'}

In [ ]:
df = conditionStatistics_5x4Expt2.xs([4.0, .57],level = ['alpha', 'signalTypePriors'])

In [ ]:
df

In [ ]:
drawPerformanceBarPlot(df, targetMeansExp2, targetSEExp2, legendColors, 8, hatch)